In [1]:
import os
import numpy as np
from utils import *
import pandas as pd

/home/forrest/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [6]:
# from batch ID to the corresponding sample IDs
skip_samples = {
    5: range(40, 50), 
    12: range(21, 30), 
    15: range(41, 50)
}

In [10]:
agreement_results = []
result_path = 'batch_5_src_no_sports/results'
label_maps = [
    {'consistent':2, 'questionable':1, 'unwanted':0, 'benign': 0},
    {'consistent':2, 'questionable':1, 'benign': 1, 'unwanted':0},
    {'consistent':2, 'benign': 2, 'questionable':1, 'unwanted':0},
    {'consistent':2, 'benign': 1, 'questionable':np.nan, 'unwanted':0},
    {'consistent':1, 'benign': 1, 'questionable':0, 'unwanted':0},
    {'consistent':1, 'benign': 0, 'questionable':0, 'unwanted':0},
    {'consistent':1, 'benign': np.nan, 'questionable':np.nan, 'unwanted':0},
]
for batch_id in range(1, 16+1):
    file_path = os.path.join(result_path, f"batch_{batch_id}_annotation.json")
    # For batch 5 (3005), skip samples 41-50 as the summaries are very identical to those in 21-30
    # skip_sample_ids = []
    # if batch_id == 5: 
    #     skip_sample_ids = [str(s_id) for s_id in range(40, 50)]
    if batch_id in skip_samples:
        skip_sample_ids = [str(s_id) for s_id in skip_samples[batch_id]]
        print (f"Skipping samples {skip_sample_ids}")
    selected_annotators = None
    # there is an unexpected "new" annotator in batch 7
    if batch_id == 7:
        selected_annotators = ['yujia', 'manveer']

    agreement_local_batch = []
    for label_map in label_maps:
        agreement_num = compute_interannotator_agreement(file_path=file_path, label_map=label_map, selected_annotators=selected_annotators, skip_sample_ids=skip_sample_ids)
        agreement_local_batch.append(agreement_num)
        
    agreement_results.append(agreement_local_batch)

batch_5_src_no_sports/results/batch_1_annotation.json
annotator for agreement computation: ['forrest', 'renyi']
disagreed sample ids in batch: [14, 15, 16, 17, 19, 23]
forrest [0, 0, 0, 0, 0, 0]
renyi [2, 2, 2, 2, 2, 2]
interval Krippendorff's alpha for label map
{'consistent': 2, 'questionable': 1, 'unwanted': 0, 'benign': 0}
0.759
batch_5_src_no_sports/results/batch_1_annotation.json
annotator for agreement computation: ['forrest', 'renyi']
disagreed sample ids in batch: [13, 14, 15, 16, 17, 19, 23]
forrest [1, 0, 0, 1, 1, 1, 1]
renyi [0, 2, 2, 2, 2, 2, 2]
interval Krippendorff's alpha for label map
{'consistent': 2, 'questionable': 1, 'benign': 1, 'unwanted': 0}
0.858
batch_5_src_no_sports/results/batch_1_annotation.json
annotator for agreement computation: ['forrest', 'renyi']
disagreed sample ids in batch: [5, 13, 14, 15]
forrest [1, 2, 0, 0]
renyi [2, 0, 2, 2]
interval Krippendorff's alpha for label map
{'consistent': 2, 'benign': 2, 'questionable': 1, 'unwanted': 0}
0.87
batch_5

In [8]:
# pd display only 3 digits
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.DataFrame(agreement_results, index=range(1, 16+1)) 

,0,1,2,3,4,5,6
1,0.759,0.858,0.870,0.858,0.841,0.759,0.908
2,0.610,0.665,0.593,0.724,0.600,0.639,0.824
3,0.571,0.596,0.474,0.622,0.417,0.569,0.666
4,0.672,0.629,0.524,0.665,0.520,0.682,0.699
5,0.493,0.499,0.429,0.613,0.308,0.396,0.686
6,0.629,0.691,0.663,0.716,0.603,0.580,0.783
7,0.878,0.653,0.309,0.747,0.320,0.891,1.000
8,0.392,0.363,0.345,0.451,0.415,0.474,0.455
9,0.441,0.563,0.564,0.598,0.522,0.382,0.683
10,0.349,0.424,0.409,0.457,0.287,0.238,0.552
